In [2]:
import sqlite3

In [3]:
db_path = "/Users/jjpr/dev/mkgu/lookup.db"

In [4]:
conn = sqlite3.connect(db_path)

In [5]:
conn

In [6]:
cursor = conn.cursor()

In [7]:
cursor

In [8]:
sql_create_store = """CREATE TABLE store 
    (id integer primary key, type text, location text);"""

In [9]:
sql_create_assembly = """CREATE TABLE assembly 
    (id integer primary key, name text unique);"""

In [10]:
sql_create_assembly_store = """CREATE TABLE assembly_store 
    (id integer primary key, role text, 
    store_id integer, assembly_id integer);"""

In [ ]:
sql_last = "SELECT last_insert_rowid();"

In [11]:
cursor.execute(sql_create_store)

In [12]:
cursor.execute(sql_create_assembly)

In [13]:
cursor.execute(sql_create_assembly_store)

In [16]:
sql_insert_store = """INSERT INTO store 
(type, location) 
VALUES 
("S3", 
"https://s3.amazonaws.com/mkgu-dicarlolab-hvm/hvm_neuronal_features.nc")
"""

In [17]:
cursor.execute(sql_insert_store)
store_id = cursor.lastrowid
store_id

1

In [18]:
sql_insert_assy = """INSERT INTO assembly 
(name) 
VALUES 
("HvM")
"""

In [19]:
cursor.execute(sql_insert_assy)
assy_id = cursor.lastrowid
assy_id

1

In [20]:
sql_insert_assy_store = """INSERT INTO assembly_store 
(role, store_id, assembly_id) 
VALUES 
("HvM", ?, ?)
"""

In [21]:
cursor.execute(sql_insert_assy_store, (store_id, assy_id))
assy_store_id = cursor.lastrowid
assy_store_id

1

In [22]:
conn.commit()

In [23]:
conn.close()